In [3]:
import os
from pathlib import Path

print("CWD:", os.getcwd())

p_parq = Path("./03-futures_prices_changes/norgate_continuous/daily_changes/parquet")
p_csv  = Path("./03-futures_prices_changes/norgate_continuous/daily_changes/csv")

print("PARQUET DIR exists?", p_parq.exists(), "->", p_parq.resolve())
print("CSV DIR exists?   ", p_csv.exists(),  "->", p_csv.resolve())

if p_parq.exists():
    print("Parquet files:", len(list(p_parq.glob("*.parquet"))))
    print("Example:", list(p_parq.glob("*.parquet"))[:5])

if p_csv.exists():
    print("CSV files:", len(list(p_csv.glob("*.csv"))))
    print("Example:", list(p_csv.glob("*.csv"))[:5])


CWD: c:\TWS API\source\pythonclient\TradingIdeas\Futures
PARQUET DIR exists? False -> C:\TWS API\source\pythonclient\TradingIdeas\Futures\03-futures_prices_changes\norgate_continuous\daily_changes\parquet
CSV DIR exists?    False -> C:\TWS API\source\pythonclient\TradingIdeas\Futures\03-futures_prices_changes\norgate_continuous\daily_changes\csv


In [4]:
#!/usr/bin/env python3
import os
from pathlib import Path
import pandas as pd
import numpy as np

"""
===============================================================================
ADD STANDARD DEVIATION MEASURES TO PER-INSTRUMENT CONTINUOUS FUTURES FILES
===============================================================================

Reads per-instrument files that already include:
- daily_prices_change_pts
- daily_prices_change_percent

Computes (per instrument):
1) daily_std_full              = std of daily returns over full dataset
2) annualized_std_full         = daily_std_full * sqrt(TRADING_DAYS_PER_YEAR)
3) ewma32_std                  = sqrt(EWMA(mean of r^2)) with alpha=lambda, min_periods=32

Writes new per-instrument outputs:
- CSV:    OUTPUT_ROOT/csv/<SYMBOL>.csv
- Parquet OUTPUT_ROOT/parquet/<SYMBOL>.parquet
"""

# ============================================================
# Configuration
# ============================================================

# Input folders (these should point to the outputs you created after adding daily changes)
# Adjust to match your actual folder names.
INPUT_PARQUET_DIR = Path("./03-futures_price_changes/norgate_continuous/daily_changes/parquet")
INPUT_CSV_DIR     = Path("./03-futures_price_changes/norgate_continuous/daily_changes/csv")
PREFER_PARQUET_INPUT = True

# Output folders (new set of files; does not overwrite input)
OUTPUT_ROOT = Path("./04-futures_price_volatility/norgate_continuous/volatility_measures")
OUTPUT_CSV_DIR = OUTPUT_ROOT / "csv"
OUTPUT_PARQUET_DIR = OUTPUT_ROOT / "parquet"

# Trading days for annualization
TRADING_DAYS_PER_YEAR = 256

# EWMA settings
EWMA_MIN_PERIODS = 32
EWMA_LAMBDA = 0.06061  # smoothing factor alpha in pandas ewm(alpha=...)

# Required columns (canonical names we will enforce)
DATE_COL = "date"
RET_COL  = "daily_prices_change_percent"

# Optional column aliases to harmonize common variants
COL_ALIASES = {
    "delivery month": ["delivery month", "delivery_month", "deliverymonth", "contract_month", "delivery"],
    "open interest":  ["open interest", "open_interest", "openinterest", "oi"],
}

# New columns to add
NEW_COLS = ["daily_std_full", "annualized_std_full", "ewma32_std"]


# ============================================================
# Implementation
# ============================================================

def ensure_dirs() -> None:
    OUTPUT_CSV_DIR.mkdir(parents=True, exist_ok=True)
    OUTPUT_PARQUET_DIR.mkdir(parents=True, exist_ok=True)

def normalize_columns(df: pd.DataFrame) -> pd.DataFrame:
    df.columns = [str(c).strip().lower() for c in df.columns]
    return df

def ensure_date_column(df: pd.DataFrame) -> pd.DataFrame:
    # If date is index, bring it back as a column
    if isinstance(df.index, pd.DatetimeIndex) and (DATE_COL not in df.columns):
        df = df.reset_index()

    if DATE_COL not in df.columns:
        raise ValueError(f"Missing '{DATE_COL}' column. Found: {list(df.columns)}")

    df[DATE_COL] = pd.to_datetime(df[DATE_COL], errors="coerce")
    df = df.dropna(subset=[DATE_COL]).sort_values(DATE_COL)
    return df

def harmonize_optional_cols(df: pd.DataFrame) -> pd.DataFrame:
    # Ensure canonical optional columns exist (even if NaN)
    for canonical, aliases in COL_ALIASES.items():
        found = None
        for a in aliases:
            a = a.lower()
            if a in df.columns:
                found = a
                break
        if found is None:
            df[canonical] = np.nan
        elif found != canonical:
            df = df.rename(columns={found: canonical})
    return df

def add_vol_measures(df: pd.DataFrame) -> pd.DataFrame:
    if RET_COL not in df.columns:
        raise ValueError(
            f"Missing '{RET_COL}' column. "
            f"Ensure you ran the daily change script first. Found: {list(df.columns)}"
        )

    r = pd.to_numeric(df[RET_COL], errors="coerce")

    # 1) Daily std over full dataset (ignore NaNs)
    daily_std_full = float(r.std(ddof=1)) if r.notna().sum() >= 2 else np.nan

    # 2) Annualized std (256 trading days)
    annualized_std_full = float(daily_std_full * np.sqrt(TRADING_DAYS_PER_YEAR)) if pd.notna(daily_std_full) else np.nan

    # 3) EWMA(32) std from squared returns; start after 32 observations
    # Using ewm mean on r^2, then sqrt -> std
    r2 = r ** 2
    ewma_var = r2.ewm(alpha=EWMA_LAMBDA, adjust=False, min_periods=EWMA_MIN_PERIODS).mean()
    ewma32_std = np.sqrt(ewma_var)

    df["daily_std_full"] = daily_std_full
    df["annualized_std_full"] = annualized_std_full
    df["ewma32_std"] = ewma32_std

    return df

def reorder_columns(df: pd.DataFrame) -> pd.DataFrame:
    base = [
        "date", "open", "high", "low", "close", "volume",
        "delivery month", "open interest",
        "daily_prices_change_pts", "daily_prices_change_percent",
        "daily_std_full", "annualized_std_full", "ewma32_std",
    ]
    present_base = [c for c in base if c in df.columns]
    remaining = [c for c in df.columns if c not in present_base]
    return df[present_base + remaining]

def read_symbol(sym: str) -> pd.DataFrame:
    pq = INPUT_PARQUET_DIR / f"{sym}.parquet"
    csv = INPUT_CSV_DIR / f"{sym}.csv"

    if PREFER_PARQUET_INPUT and pq.exists():
        df = pd.read_parquet(pq)
    elif csv.exists():
        df = pd.read_csv(csv)
    elif pq.exists():
        df = pd.read_parquet(pq)
    else:
        raise FileNotFoundError(f"No input found for {sym} in {INPUT_PARQUET_DIR} or {INPUT_CSV_DIR}")

    df = normalize_columns(df)
    df = ensure_date_column(df)
    df = harmonize_optional_cols(df)
    return df

def write_symbol(sym: str, df: pd.DataFrame) -> None:
    out_csv = OUTPUT_CSV_DIR / f"{sym}.csv"
    out_pq  = OUTPUT_PARQUET_DIR / f"{sym}.parquet"
    df.to_csv(out_csv, index=False)
    df.to_parquet(out_pq, index=False, compression="snappy")

def main() -> None:
    ensure_dirs()

    syms = set()
    if INPUT_PARQUET_DIR.exists():
        syms |= {p.stem for p in INPUT_PARQUET_DIR.glob("*.parquet")}
    if INPUT_CSV_DIR.exists():
        syms |= {p.stem for p in INPUT_CSV_DIR.glob("*.csv")}

    if not syms:
        raise RuntimeError(
            f"No per-instrument files found.\n"
            f"Checked:\n- {INPUT_PARQUET_DIR}\n- {INPUT_CSV_DIR}"
        )

    print(f"Found {len(syms)} instruments.")
    for sym in sorted(syms):
        df = read_symbol(sym)
        df = add_vol_measures(df)
        df = reorder_columns(df)
        write_symbol(sym, df)

    print(f"Done. Wrote new files to: {OUTPUT_ROOT}")

if __name__ == "__main__":
    main()


Found 105 instruments.
Done. Wrote new files to: 04-futures_price_volatility\norgate_continuous\volatility_measures
